In [1]:
import time
import numpy as np
import pandas as pd
import joblib
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import GradientBoostingRegressor, RandomForestRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error

In [2]:
# 1. Wczytanie danych
# Pobieramy publiczny zbiór danych California Housing, który zawiera informacje o cenach nieruchomości w Kalifornii.
data = fetch_california_housing()
df = pd.DataFrame(data.data, columns=data.feature_names)
df['target'] = data.target  # Dodajemy kolumnę z cenami mieszkań (wartość docelowa)


In [3]:
# 2. Podział na zbiór treningowy i testowy
# Dane dzielimy na 80% do treningu i 20% do testów, aby móc ocenić model na nieznanych danych.
X_train, X_test, y_train, y_test = train_test_split(df.drop(columns=['target']), df['target'], test_size=0.2,
                                                    random_state=42)

In [4]:
# 3. Skalowanie cech
# Normalizujemy dane, aby poprawić efektywność algorytmów uczenia maszynowego.
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [5]:
# 4. Inicjalizacja modeli
# Wybieramy dwa modele:
# - Gradient Boosting: silny model boostingowy, dobrze radzący sobie na zbiorach danych o złożonych zależnościach.
# - Random Forest: las losowy, który dobrze generalizuje, ale może wymagać większych zasobów obliczeniowych.
models = {
    "Gradient Boosting": GradientBoostingRegressor(n_estimators=500, learning_rate=0.1, max_depth=4, random_state=42),
    "Random Forest": RandomForestRegressor(n_estimators=500, max_depth=10, random_state=42, n_jobs=-1)
}

# Parametry:
# - n_estimators: liczba drzew decyzyjnych w modelu (500 dla obu modeli).
# - learning_rate (tylko dla Gradient Boosting): tempo uczenia (0.1).
# - max_depth: maksymalna głębokość drzewa decyzyjnego (4 dla Gradient Boosting, 10 dla Random Forest).
# - random_state: zapewnia powtarzalność wyników.
# - n_jobs (-1 dla Random Forest): używa wszystkich dostępnych rdzeni procesora.

In [6]:
# 5. Trenowanie i ewaluacja
# Dla każdego modelu mierzymy czas trenowania i obliczamy błędy MAE oraz RMSE.
results = {}
for name, model in models.items():
    start_time = time.time()
    model.fit(X_train_scaled, y_train)
    train_time = time.time() - start_time

    y_pred = model.predict(X_test_scaled)
    mae = mean_absolute_error(y_test, y_pred)  # Średni błąd bezwzględny
    rmse = np.sqrt(mean_squared_error(y_test, y_pred))  # Pierwiastek z błędu średniokwadratowego

    results[name] = {
        "Train Time (s)": train_time,
        "MAE": mae,
        "RMSE": rmse
    }

    joblib.dump(model, f"{name}_model.pkl")  # Zapisanie modelu do pliku

In [7]:
# 6. Wyświetlenie wyników
# Wyniki przedstawiamy w formie tabeli zawierającej czas trenowania oraz wartości błędów.
results_df = pd.DataFrame(results).T
print(results_df)

                   Train Time (s)       MAE      RMSE
Gradient Boosting       45.131116  0.308304  0.467761
Random Forest           38.581659  0.365239  0.542726


In [8]:
#7 predykcja na danych zewnetrznych
sample_data = np.mean(X_test_scaled, axis=0).reshape(1,-1)  # Średnie wartości cech ze zbioru testowego
sample_data_scaled = scaler.transform(sample_data)
print("\nPrzykladowe dane wejsciowe:")
print(sample_data)


Przykladowe dane wejsciowe:
[[-0.02647585  0.01237949 -0.01305925 -0.00011079 -0.00429306 -0.01135997
  -0.0264153   0.03137725]]


/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


In [9]:
print(pd.DataFrame(sample_data,columns=data.feature_names))

     MedInc  HouseAge  AveRooms  AveBedrms  Population  AveOccup  Latitude  \
0 -0.026476  0.012379 -0.013059  -0.000111   -0.004293  -0.01136 -0.026415   

   Longitude  
0   0.031377  


In [10]:
for name, model in models.items():
    prediction = model.predict(sample_data_scaled)
    print(f"\nPredykcja dla modelu {name}: {prediction[0]:.2f} [setki tysięcy $]")


Predykcja dla modelu Gradient Boosting: 5.42 [setki tysięcy $]

Predykcja dla modelu Random Forest: 1.73 [setki tysięcy $]
